In [1]:
import os
from tqdm import tqdm
from glob import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
p = "./data/train.csv"
df = pd.read_csv(p)
df

,image_id,label,image_width,image_height,is_tma
0,4,HGSC,23785,20008,False
1,66,LGSC,48871,48195,False
2,91,HGSC,3388,3388,True
3,281,LGSC,42309,15545,False
4,286,EC,37204,30020,False
...,...,...,...,...,...
533,65022,LGSC,53355,46675,False
534,65094,MC,55042,45080,False
535,65300,HGSC,75860,27503,False
536,65371,HGSC,42551,41800,False


In [45]:
isinstance(df["is_tma"].values.tolist()[0], bool)

True

## Class 종류 및 개수

In [9]:
p = "./data/train.csv"
df = pd.read_csv(p)
df['label'].value_counts()

HGSC    222
EC      124
CC       99
LGSC     47
MC       46
Name: label, dtype: int64

## 클래스 별 데이터 시각화
- 크기를 512 x 512 로 줄여서 진행

In [ ]:
import matplotlib.pyplot as plt

root_path = "./data/train_thumbnails"
p = "./data/train.csv"
df = pd.read_csv(p)

f, axs = plt.subplots(5,5, figsize=(50,50))


label_name = df['label'].unique()
for i, ln in enumerate(label_name) :
    label_list = df['image_id'][df['label'] == ln].values
    
    while 1:
        pick_num = np.random.choice(len(label_list), 5, replace=False)
        pick_label = label_list[pick_num]
        flag = False
        for j, num in enumerate(pick_label) :
            img_path = os.path.join(root_path, str(num)+"_thumbnail.png")
            if os.path.exists(img_path) :
                flag = True
            else :
                flag = False
                break
        
    for j, num in enumerate(pick_label) :
        img_path = os.path.join(root_path, str(num)+"_thumbnail.png")
        
        if os.path.exists(img_path) :
            img = cv2.imread(img_path)
            img = cv2.resize(img, (512, 512))
            img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
            
            axs[i][j].imshow(img)
            axs[i][j].set_title(f"{ln} : {num}_thumbnail.png")
            
        else :
            continue
        

In [18]:
df[df['label'] == 'CC']['image_id'].values

array([ 1660,  1943,  1952,  3191,  4827,  4877,  5970,  6359,  6449,
        7482,  7955,  8279,  9658, 10548, 11263, 12222, 12442, 13526,
       13987, 14051, 15231, 15470, 15486, 16325, 16876, 17067, 17291,
       18014, 18813, 20205, 21929, 22155, 22290, 22425, 22489, 24563,
       25923, 25928, 26219, 26533, 26950, 27315, 28603, 29615, 29915,
       30203, 30272, 30369, 31033, 32032, 35197, 36063, 36302, 38349,
       38669, 38687, 39880, 40129, 40888, 41586, 42125, 42296, 42549,
       42857, 43796, 44283, 44581, 44603, 44976, 45725, 45990, 46543,
       46769, 47960, 48506, 51346, 52259, 52375, 52420, 52461, 52931,
       53377, 54928, 55287, 56861, 56947, 57598, 57696, 58947, 58974,
       59515, 60685, 60988, 61961, 62828, 63015, 63165, 63289, 64824],
      dtype=int64)

## 일정 pixel 값 이상이면 0으로 채워서 배경삭제

In [85]:
root_path = "./data/train_thumbnails/"
fn = "_thumbnail.png"
p = "./data/train.csv"
df = pd.read_csv(p)
no_tma_df = df[df["is_tma"] == False]

sampling_num = np.random.choice(len(no_tma_df), 5, replace=False)
sampled_df = no_tma_df.iloc[sampling_num]

In [ ]:
f, axs = plt.subplots(5,6, figsize=(50,50))
for i in range(6) :
    for idx, _df in enumerate(sampled_df.iloc) :
        path = os.path.join(root_path, str(_df['image_id'])+fn)
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
        ori_shape = img.shape[:2]
        
        if i == 0 : 
            img[np.where((img > [200, 200, 200]).all(axis = 2))] = [0,0,0]
            t = 200
        elif i == 1 :
            img[np.where((img > [215, 215, 215]).all(axis = 2))] = [0,0,0]
            t = 215
        elif i == 2 :
            img[np.where((img > [225, 225, 225]).all(axis = 2))] = [0,0,0]
            t = 225
        elif i == 3 :
            img[np.where((img > [235, 235, 235]).all(axis = 2))] = [0,0,0]
            t = 235
        elif i == 4 :
            img[np.where((img > [245, 245, 245]).all(axis = 2))] = [0,0,0]
            t = 255
        
        img = cv2.resize(img, (512, 512))
        axs[idx][i].imshow(img)
        axs[idx][i].set_title("threshold : " + str(t))
plt.show()
    

## 1개의 이미지 내에 여러개의 객체가 있는 이미지 분할 하기

In [7]:
root_path = "./data/train_thumbnails/"
fn = "_thumbnail.png"
p = "./data/train.csv"
df = pd.read_csv(p)
no_tma_df = df[df["is_tma"] == False]

sampling_num = np.random.choice(len(no_tma_df), 5, replace=False)
sampled_df = no_tma_df.iloc[sampling_num]

In [ ]:
sampled_df = [706, 1660, 1666, 2706, 1943, 1774, 46444, 48502, 61100, 59031]
f, axs = plt.subplots(len(sampled_df), 6, figsize=(50,50))
for i, _df in enumerate(sampled_df) :
    path = os.path.join(root_path, str(_df)+fn)
    img = cv2.imread(path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, binary = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    cropped_images= []
    idx = 0
    for j, contour in enumerate(contours):
        x, y, w, h = cv2.boundingRect(contour)
        cropped_image = img[y:y+h, x:x+w]
        cropped_image[np.where((cropped_image > [215, 215, 215]).all(axis = 2))] = [0,0,0]
        # if w > 200 and h > 200 :
        if np.sum(np.array(cropped_image))>10000000:
            # cropped_images.append(cropped_image)
            h, w = cropped_image.shape[:2]
            cropped_image = cv2.resize(cropped_image, (512, 512))
            axs[i][idx].imshow(cropped_image)
            axs[i][idx].set_title(str(_df) + " >> w : " + str(w) + " // h : "+str(h))
            idx += 1
            
            
plt.show()

In [2]:
a = [1,2,3]
b = [5,6,7]

a = a+b
a

[1, 2, 3, 5, 6, 7]

In [6]:
from glob import glob
import os

p = "./ckpt/tf_efficientnetv2_s.in21k/rmbg_lossweight_effiv2s_512/*.pth"
sorted(glob(p), key=lambda x:int(os.path.basename(x).split("E-")[0]))

['./ckpt/tf_efficientnetv2_s.in21k/rmbg_lossweight_effiv2s_512\\0E-val0.3833333333333334-tf_efficientnetv2_s.in21k.pth',
 './ckpt/tf_efficientnetv2_s.in21k/rmbg_lossweight_effiv2s_512\\2E-val0.3903846153846154-tf_efficientnetv2_s.in21k - 복사본.pth',
 './ckpt/tf_efficientnetv2_s.in21k/rmbg_lossweight_effiv2s_512\\3E-val0.3903846153846154-tf_efficientnetv2_s.in21k.pth',
 './ckpt/tf_efficientnetv2_s.in21k/rmbg_lossweight_effiv2s_512\\10E-val0.3903846153846154-tf_efficientnetv2_s.in21k - 복사본 - 복사본.pth']

In [8]:
import timm
timm.list_models("*efficient*", pretrained=False)

['efficientformer_l1',
 'efficientformer_l3',
 'efficientformer_l7',
 'efficientformerv2_l',
 'efficientformerv2_s0',
 'efficientformerv2_s1',
 'efficientformerv2_s2',
 'efficientnet_b0',
 'efficientnet_b0_g8_gn',
 'efficientnet_b0_g16_evos',
 'efficientnet_b0_gn',
 'efficientnet_b1',
 'efficientnet_b1_pruned',
 'efficientnet_b2',
 'efficientnet_b2_pruned',
 'efficientnet_b2a',
 'efficientnet_b3',
 'efficientnet_b3_g8_gn',
 'efficientnet_b3_gn',
 'efficientnet_b3_pruned',
 'efficientnet_b3a',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_b8',
 'efficientnet_cc_b0_4e',
 'efficientnet_cc_b0_8e',
 'efficientnet_cc_b1_8e',
 'efficientnet_el',
 'efficientnet_el_pruned',
 'efficientnet_em',
 'efficientnet_es',
 'efficientnet_es_pruned',
 'efficientnet_l2',
 'efficientnet_lite0',
 'efficientnet_lite1',
 'efficientnet_lite2',
 'efficientnet_lite3',
 'efficientnet_lite4',
 'efficientnetv2_l',
 'efficientnetv2_m',
 'efficientnetv2_rw_m',
 'efficie